# Dementia and Pills Plots

In [1]:
import pandas as pd
import numpy as np

from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 7]

In [2]:
import stata_setup

stata_setup.config("/Applications/Stata", "mp")


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core , expiring 20 Jan 2022
Serial number: 501709220007
  Licensed to: Edie Espejo
               

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [3]:
%%stata
clear

In [4]:
%%stata
use "../data/step-2/cleaned-data.dta"
replace subpop=0 if binaryattitude3==. & binaryattitude4==. & pillsmax==.

svyset w6varunit [pweight=w6anfinwgt0], strata(w6varstrat)

recode is6prxyrelat (6/30 32/92 =6), gen (prxyrelat_new)
label variable prxyrelat_new "proxy-relationship"
label define prxyrelat 2 "spouse/partner" 3 "daughter" 4 "son" 5 "daughter-in-law" 31 "paid aide/employee" 6 "other"
label values prxyrelat_new prxyrelat

label variable is6famrrutin "how familiar with routine"
label define is6famrrutin 1 "very familiar" 2 "somewhat familiar" 3 "a little familiar" 4 "not at all/refused/don't know"
label values is6famrrutin is6famrrutin


. use "../data/step-2/cleaned-data.dta"
(Written by R)

. 
. svyset w6varunit [pweight=w6anfinwgt0], strata(w6varstrat)

Sampling weights: w6anfinwgt0
             VCE: linearized
     Single unit: missing
        Strata 1: w6varstrat
 Sampling unit 1: w6varunit
           FPC 1: <zero>

. 
. recode is6prxyrelat (6/30 32/92 =6), gen (prxyrelat_new)
(18 differences between is6prxyrelat and prxyrelat_new)

. label variable prxyrelat_new "proxy-relationship"

. label define prxyrelat 2 "spouse/partner" 3 "daughter" 4 "son" 5 "daughter-in
> -law" 31 "paid aide/employee" 6 "other"

. label values prxyrelat_new prxyrelat

. 
. label variable is6famrrutin "how familiar with routine"

. label define is6famrrutin 1 "very familiar" 2 "somewhat familiar" 3 "a little
>  familiar" 4 "not at all/refused/don't know"

. label values is6famrrutin is6famrrutin

. 


In [5]:
%%stata

gen dementia_diagnosis=.
replace dementia_diagnosis=0 if hc6disescn9==0
replace dementia_diagnosis=1 if hc6disescn9==7
replace dementia_diagnosis=1 if hc6disescn9==1

label variable dementia_diagnosis "diagnosed with dementia"
label define dementia_diagnosis_lab 0 "not diagnosed" 1 "diagnosed"
label values dementia_diagnosis dementia_diagnosis_lab


. 
. gen dementia_diagnosis=.
(2,096 missing values generated)

. replace dementia_diagnosis=0 if hc6disescn9==0
(1,935 real changes made)

. replace dementia_diagnosis=1 if hc6disescn9==7
(0 real changes made)

. replace dementia_diagnosis=1 if hc6disescn9==1
(161 real changes made)

. 
. label variable dementia_diagnosis "diagnosed with dementia"

. label define dementia_diagnosis_lab 0 "not diagnosed" 1 "diagnosed"

. label values dementia_diagnosis dementia_diagnosis_lab

. 


In [6]:
%%stata
gen pillsmax_clean = .
replace pillsmax_clean=ma6pillsmax if ma6pillsmax>0

quietly: tabout pillsmax_clean if subpop using "../tables/plottables/pillsmax.csv", svy cells(cells) format(2) replace
    
quietly: tabout dementia_diagnosis pillsmax_clean if subpop using "../tables/plottables/diagnosis-pills.csv", svy cells(row) format(2) replace
quietly: tabout dementia_diagnosis pillsmax_clean if subpop using "../tables/plottables/diagnosis-pills-2.csv", svy cells(cells) format(2) replace


. gen pillsmax_clean = .
(2,096 missing values generated)

. replace pillsmax_clean=ma6pillsmax if ma6pillsmax>0
(1,982 real changes made)

. 
. quietly: tabout pillsmax_clean if subpop using "../tables/plottables/pillsmax
> .csv", svy cells(cells) format(2) replace

.     
. quietly: tabout dementia_diagnosis pillsmax_clean if subpop using "../tables/
> plottables/diagnosis-pills.csv", svy cells(row) format(2) replace

. quietly: tabout dementia_diagnosis pillsmax_clean if subpop using "../tables/
> plottables/diagnosis-pills-2.csv", svy cells(cells) format(2) replace

. 


In [7]:
pills = pd.read_table('../tables/plottables/pillsmax.csv')
pills

,pillsmax_clean,Prop.
0,1,0.50
1,2,0.21
2,3,0.15
3,4,0.09
4,5,0.03
5,6,0.02
6,Total,1.00


In [8]:
diagnosis_pills_row = pd.read_table('../tables/plottables/diagnosis-pills.csv', skiprows=1)
diagnosis_pills_row = diagnosis_pills_row.drop([0])
diagnosis_pills_row

,diagnosed with dementia,1,2,3,4,5,6,Total
1,not diagnosed,0.52,0.21,0.15,0.07,0.02,0.03,1.00
2,diagnosed,0.48,0.20,0.15,0.12,0.04,0.01,1.00
3,Total,0.50,0.21,0.15,0.09,0.03,0.02,1.00


In [9]:
diagnosis_pills_cell = pd.read_table('../tables/plottables/diagnosis-pills-2.csv', skiprows=1)
diagnosis_pills_cell = diagnosis_pills_cell.drop([0])
diagnosis_pills_cell

,diagnosed with dementia,1,2,3,4,5,6,Total
1,not diagnosed,0.34,0.14,0.10,0.05,0.01,0.02,0.66
2,diagnosed,0.16,0.07,0.05,0.04,0.01,0.00,0.34
3,Total,0.50,0.21,0.15,0.09,0.03,0.02,1.00


In [10]:
%%stata
quietly: tabout dementia pillsmax_clean if subpop using "../tables/plottables/dementia-pills.csv", svy cells(row) format(2) replace
quietly: tabout dementia pillsmax_clean if subpop using "../tables/plottables/dementia-pills-2.csv", svy cells(cells) format(2) replace


. quietly: tabout dementia pillsmax_clean if subpop using "../tables/plottables
> /dementia-pills.csv", svy cells(row) format(2) replace

. quietly: tabout dementia pillsmax_clean if subpop using "../tables/plottables
> /dementia-pills-2.csv", svy cells(cells) format(2) replace

. 


In [11]:
dementia_pills_row = pd.read_table('../tables/plottables/dementia-pills.csv', skiprows=1)
dementia_pills_row = dementia_pills_row.drop([0])
dementia_pills_row

,dementia status,1,2,3,4,5,6,Total
1,possible dementia,0.50,0.22,0.17,0.08,0.02,0.01,1.00
2,probable dementia,0.50,0.20,0.13,0.10,0.03,0.03,1.00
3,Total,0.50,0.21,0.15,0.09,0.03,0.02,1.00


In [12]:
dementia_pills_cell = pd.read_table('../tables/plottables/dementia-pills-2.csv', skiprows=1)
dementia_pills_cell = dementia_pills_cell.drop([0])
dementia_pills_cell

,dementia status,1,2,3,4,5,6,Total
1,possible dementia,0.23,0.10,0.08,0.04,0.01,0.01,0.45
2,probable dementia,0.28,0.11,0.07,0.06,0.02,0.02,0.55
3,Total,0.50,0.21,0.15,0.09,0.03,0.02,1.00


In [13]:
pills.to_csv('../tables/clean-csv/pills.csv')

diagnosis_pills_row.to_csv('../tables/clean-csv/diagnosis-pills-row.csv')
diagnosis_pills_cell.to_csv('../tables/clean-csv/diagnosis-pills-cell.csv')
dementia_pills_row.to_csv('../tables/clean-csv/dementia-pills-row.csv')
dementia_pills_cell.to_csv('../tables/clean-csv/dementia-pills-cell.csv')